In [278]:
#Calculate MAE on ORIGINAL 
from fbm import fbm, fgn, times
import numpy as np
from ipynb.fs.defs.Fractool_Original import FracTool
import sklearn
from sklearn.metrics import mean_absolute_error

def MSE(N1,N2,N3):
    '''This function generates a matrix of known Hurst values and calculates the MSE of the expected Hurst and the 
    actual Hurst outputted by FracTool. The leftmost third of the matrix are fGn signals, the middle third are fBm 
    signals, and the last third are random arrays. 
    N1 = number of rows in matrix
    N2 = number of columns in matrix
    N3 = number of time-points in signal (3rd dimension in matrix)
    '''
    
    #first generate an matrix with known Hurst values and signal class
    
    #create arrays of expected Hurst and class (if interested in class as well)
    Hurst_expected = np.zeros((N1,N2))
    Sig_class_expected = np.zeros((N1,N2))
    Signal_matrix = np.zeros((N1,N2,N3))
    fgn_section = np.floor(N2/3) #end of leftmost third of columns of matrix
    fbm_section = fgn_section + fgn_section #end of middle third of columns of matrix
   
    for i in range(N1):
        #leftmost third of matrix (fGn signals)
        for j in range(0,int(fgn_section)):
            Hurst_fgn = random.random() #generates random number between 0 and 1 (random Hurst value)
            Hurst_expected[i][j] = Hurst_fgn
            Sig_class_expected[i][j] = 0 #fGn class
            #generate fGn signal with random Hurst at each element in first third of matrix
            Signal_matrix[i][j] = fgn(n=N3, hurst = Hurst_fgn, length=1, method='daviesharte')
        
        #middle third of matrix (fBm signals)
        for k in range(int(fgn_section),int(fbm_section)):
            Hurst_fbm = random.random()
            Hurst_expected[i][k] = Hurst_fbm
            Sig_class_expected[i][k] = 2 #fBm class
            #generate fBm signal with random Hurst at each element in middle third of matrix
            Signal_matrix[i][k] = fbm(n=N3+1, hurst = Hurst_fbm, length=1, method='daviesharte')[1:N3+1]
        #rightmost third of matrix (random signals)
        for l in range(int(fbm_section), N2):
            Hurst_rand = 0 #expected Hurst for random signal?
            Hurst_expected[i][l] = Hurst_rand
            Sig_class_expected[i][l] = 3 #outside model
            #generate random array (values between 0 and 1) of length N3 at each element in last third of matrix
            Signal_matrix[i][l] = list(np.random.rand(N3))
    
    #Now find the Hurst value and signal class that FracTool calculates on these known signals
    Hurst_actual = np.zeros((N1,N2))
    Sig_class_actual = np.zeros((N1,N2))
    for n in range(N1): 
        for m in range(N2):
            result = FracTool(Signal_matrix[n,m])
            #result[0] of fractool is sig class
            #result[1] of fractool is Hurst coeff
            Hurst_actual[n,m] = result[1] 
            Sig_class_actual[n,m] = result[0] #comment out if only interested in Hurst MSE
    
    
    Hurst_actual = np.nan_to_num(Hurst_actual)

    summation = 0
    misclassified_total = 0
    #if want to compute MAE of whole matrix, use range(N1) for outer for loop 
    #if want to compute MAE of fGn only, use range(int(fgn_section))
    #if want to compute MAE of fBm only, use range(int(fgn_section),int(fbm_section))
    #if want to compute MAE of random signals only, use range(int(fbm_section), N2)
    for x in range(N1):
        for y in range(N2):
            if ((Sig_class_actual[x,y] == 3 and Sig_class_expected[x,y] != 3) or (Sig_class_actual[x,y] != 3 and Sig_class_expected[x,y] == 3)): 
                #Hurst_actual[x,y] = 0
                Hurst_expected[x,y] = 1
            if ((Sig_class_actual[x,y] != Sig_class_expected[x,y])):
                misclassified_total = misclassified_total + 1
                
            diff = abs((Hurst_actual[x,y] - Hurst_expected[x,y]))
            summation = summation + diff
    
    Misclassified_Rate = misclassified_total/(N1*N2)
    MAE = summation/(N1*N2)  #need to divide by number that is included - change if only looking at specific section of matrix

    return [MAE, Misclassified_Rate]
#function call 
#ex:  N1=33, N2=33, N3 = 1024 generates 999 (33x33) signals of length = 1024 time-points
MSE(10,10,1024)

ipynb.fs.defs.Fractool_Original:87: ComplexWarning: Casting complex values to real discards the imaginary part
ipynb.fs.defs.Fractool_Original:88: ComplexWarning: Casting complex values to real discards the imaginary part


[0.3820520229254361, 0.41]

In [282]:
#Calculate MAE on ZERO_PADDING 
from fbm import fbm, fgn, times
import numpy as np
from ipynb.fs.defs.Fractool_Zero_Padding import FracTool
import sklearn
from sklearn.metrics import mean_absolute_error

def MSE(N1,N2,N3):
    '''This function generates a matrix of known Hurst values and calculates the MSE of the expected Hurst and the 
    actual Hurst outputted by FracTool. The leftmost third of the matrix are fGn signals, the middle third are fBm 
    signals, and the last third are random arrays. 
    N1 = number of rows in matrix
    N2 = number of columns in matrix
    N3 = number of time-points in signal (3rd dimension in matrix)
    '''
    
    #first generate an matrix with known Hurst values and signal class
    
    #create arrays of expected Hurst and class (if interested in class as well)
    Hurst_expected = np.zeros((N1,N2))
    Sig_class_expected = np.zeros((N1,N2))
    Signal_matrix = np.zeros((N1,N2,N3))
    fgn_section = np.floor(N2/3) #end of leftmost third of columns of matrix
    fbm_section = fgn_section + fgn_section #end of middle third of columns of matrix
   
    for i in range(N1):
        #leftmost third of matrix (fGn signals)
        for j in range(0,int(fgn_section)):
            Hurst_fgn = random.random() #generates random number between 0 and 1 (random Hurst value)
            Hurst_expected[i][j] = Hurst_fgn
            Sig_class_expected[i][j] = 0 #fGn class
            #generate fGn signal with random Hurst at each element in first third of matrix
            Signal_matrix[i][j] = fgn(n=N3, hurst = Hurst_fgn, length=1, method='daviesharte')
        
        #middle third of matrix (fBm signals)
        for k in range(int(fgn_section),int(fbm_section)):
            Hurst_fbm = random.random()
            Hurst_expected[i][k] = Hurst_fbm
            Sig_class_expected[i][k] = 2 #fBm class
            #generate fBm signal with random Hurst at each element in middle third of matrix
            Signal_matrix[i][k] = fbm(n=N3+1, hurst = Hurst_fbm, length=1, method='daviesharte')[1:N3+1]
        #rightmost third of matrix (random signals)
        for l in range(int(fbm_section), N2):
            Hurst_rand = 0 #expected Hurst for random signal?
            Hurst_expected[i][l] = Hurst_rand
            Sig_class_expected[i][l] = 3 #outside model
            #generate random array (values between 0 and 1) of length N3 at each element in last third of matrix
            Signal_matrix[i][l] = list(np.random.rand(N3))
    
    #Now find the Hurst value and signal class that FracTool calculates on these known signals
    Hurst_actual = np.zeros((N1,N2))
    Sig_class_actual = np.zeros((N1,N2))
    for n in range(N1): 
        for m in range(N2):
            result = FracTool(Signal_matrix[n,m])
            #result[0] of fractool is sig class
            #result[1] of fractool is Hurst coeff
            Hurst_actual[n,m] = result[1] 
            Sig_class_actual[n,m] = result[0] #comment out if only interested in Hurst MSE
    
    
    Hurst_actual = np.nan_to_num(Hurst_actual)

    summation = 0
    misclassified_total = 0
    #if want to compute MAE of whole matrix, use range(N1) for outer for loop 
    #if want to compute MAE of fGn only, use range(int(fgn_section))
    #if want to compute MAE of fBm only, use range(int(fgn_section),int(fbm_section))
    #if want to compute MAE of random signals only, use range(int(fbm_section), N2)
    for x in range(N1):
        for y in range(N2):
            if ((Sig_class_actual[x,y] == 3 and Sig_class_expected[x,y] != 3) or (Sig_class_actual[x,y] != 3 and Sig_class_expected[x,y] == 3)): 
                Hurst_actual[x,y] = 0
                Hurst_expected[x,y] = 1
            if ((Sig_class_actual[x,y] != Sig_class_expected[x,y])):
                misclassified_total = misclassified_total + 1
                
            diff = abs((Hurst_actual[x,y] - Hurst_expected[x,y]))
            summation = summation + diff
    
    Misclassified_Rate = misclassified_total/(N1*N2)
    MAE = summation/(N1*N2)  #need to divide by number that is included - change if only looking at specific section of matrix
    print(Sig_class_actual)
    print(Sig_class_expected)
    print(Hurst_actual)
    print(Hurst_expected)
    return [MAE, Misclassified_Rate]
#function call 
#ex:  N1=33, N2=33, N3 = 1024 generates 999 (33x33) signals of length = 1024 time-points
MSE(6,6,1024)

[[1. 0. 2. 2. 0. 0.]
 [0. 2. 2. 2. 0. 0.]
 [0. 0. 2. 2. 0. 0.]
 [1. 0. 0. 2. 0. 0.]
 [1. 0. 2. 2. 0. 0.]
 [0. 1. 2. 2. 0. 0.]]
[[0. 0. 2. 2. 3. 3.]
 [0. 0. 2. 2. 3. 3.]
 [0. 0. 2. 2. 3. 3.]
 [0. 0. 2. 2. 3. 3.]
 [0. 0. 2. 2. 3. 3.]
 [0. 0. 2. 2. 3. 3.]]
[[0.         0.70348751 0.78911312 0.74494893 0.         0.        ]
 [0.30899246 0.11573127 0.75192061 0.50354546 0.         0.        ]
 [0.67582041 0.79143674 0.45006113 0.2541812  0.         0.        ]
 [0.         0.37564591 0.92407604 0.70249027 0.         0.        ]
 [0.         0.34936645 0.25080024 0.62751108 0.         0.        ]
 [0.67165291 0.         0.72195067 0.83412714 0.         0.        ]]
[[0.76638606 0.68560688 0.93529208 0.98410405 1.         1.        ]
 [0.30691542 0.96795977 0.76320956 0.46024458 1.         1.        ]
 [0.70424745 0.70399183 0.3909669  0.16697469 1.         1.        ]
 [0.81392076 0.2797655  0.03821967 0.82758069 1.         1.        ]
 [0.72845647 0.26558332 0.21517248 0.62922636 1.       

[0.5054137524556438, 0.5]

In [274]:
#Calculate MAE on FREQ_FILTER 
from fbm import fbm, fgn, times
import numpy as np
from ipynb.fs.defs.Fractool_Freq_Filter import FracTool
import sklearn
from sklearn.metrics import mean_absolute_error

def MSE(N1,N2,N3):
    '''This function generates a matrix of known Hurst values and calculates the MSE of the expected Hurst and the 
    actual Hurst outputted by FracTool. The leftmost third of the matrix are fGn signals, the middle third are fBm 
    signals, and the last third are random arrays. 
    N1 = number of rows in matrix
    N2 = number of columns in matrix
    N3 = number of time-points in signal (3rd dimension in matrix)
    '''
    
    #first generate an matrix with known Hurst values and signal class
    
    #create arrays of expected Hurst and class (if interested in class as well)
    Hurst_expected = np.zeros((N1,N2))
    Sig_class_expected = np.zeros((N1,N2))
    Signal_matrix = np.zeros((N1,N2,N3))
    fgn_section = np.floor(N2/3) #end of leftmost third of columns of matrix
    fbm_section = fgn_section + fgn_section #end of middle third of columns of matrix
   
    for i in range(N1):
        #leftmost third of matrix (fGn signals)
        for j in range(0,int(fgn_section)):
            Hurst_fgn = random.random() #generates random number between 0 and 1 (random Hurst value)
            Hurst_expected[i][j] = Hurst_fgn
            Sig_class_expected[i][j] = 0 #fGn class
            #generate fGn signal with random Hurst at each element in first third of matrix
            Signal_matrix[i][j] = fgn(n=N3, hurst = Hurst_fgn, length=1, method='daviesharte')
        
        #middle third of matrix (fBm signals)
        for k in range(int(fgn_section),int(fbm_section)):
            Hurst_fbm = random.random()
            Hurst_expected[i][k] = Hurst_fbm
            Sig_class_expected[i][k] = 2 #fBm class
            #generate fBm signal with random Hurst at each element in middle third of matrix
            Signal_matrix[i][k] = fbm(n=N3+1, hurst = Hurst_fbm, length=1, method='daviesharte')[1:N3+1]
        #rightmost third of matrix (random signals)
        for l in range(int(fbm_section), N2):
            Hurst_rand = 0 #expected Hurst for random signal?
            Hurst_expected[i][l] = Hurst_rand
            Sig_class_expected[i][l] = 3 #outside model
            #generate random array (values between 0 and 1) of length N3 at each element in last third of matrix
            Signal_matrix[i][l] = list(np.random.rand(N3))
    
    #Now find the Hurst value and signal class that FracTool calculates on these known signals
    Hurst_actual = np.zeros((N1,N2))
    Sig_class_actual = np.zeros((N1,N2))
    for n in range(N1): 
        for m in range(N2):
            result = FracTool(Signal_matrix[n,m])
            #result[0] of fractool is sig class
            #result[1] of fractool is Hurst coeff
            Hurst_actual[n,m] = result[1] 
            Sig_class_actual[n,m] = result[0] #comment out if only interested in Hurst MSE
    
    
    Hurst_actual = np.nan_to_num(Hurst_actual)

    summation = 0
    misclassified_total = 0
    #if want to compute MAE of whole matrix, use range(N1) for outer for loop 
    #if want to compute MAE of fGn only, use range(int(fgn_section))
    #if want to compute MAE of fBm only, use range(int(fgn_section),int(fbm_section))
    #if want to compute MAE of random signals only, use range(int(fbm_section), N2)
    for x in range(N1):
        for y in range(N2):
            if ((Sig_class_actual[x,y] == 3 and Sig_class_expected[x,y] != 3) or (Sig_class_actual[x,y] != 3 and Sig_class_expected[x,y] == 3)): 
                #Hurst_actual[x,y] = 0
                Hurst_expected[x,y] = 1
            if ((Sig_class_actual[x,y] != Sig_class_expected[x,y])):
                misclassified_total = misclassified_total + 1
                
            diff = abs((Hurst_actual[x,y] - Hurst_expected[x,y]))
            summation = summation + diff
    
    Misclassified_Rate = misclassified_total/(N1*N2)
    MAE = summation/(N1*N2)  #need to divide by number that is included - change if only looking at specific section of matrix

    return [MAE, Misclassified_Rate]
#function call 
#ex:  N1=33, N2=33, N3 = 1024 generates 999 (33x33) signals of length = 1024 time-points
MSE(10,10,1024)

ipynb.fs.defs.Fractool_Freq_Filter:89: ComplexWarning: Casting complex values to real discards the imaginary part
ipynb.fs.defs.Fractool_Freq_Filter:90: ComplexWarning: Casting complex values to real discards the imaginary part


[0.4873493766043622, 0.79]